In [38]:
!ls

bank-classification.csv  experiments.ipynb  README.md  sample_submission.csv


In [64]:
!head sample_submission.csv

id,y
2,0.114428578297
4,0.114428578297
5,0.114428578297
7,0.114428578297
10,0.114428578297
12,0.114428578297
14,0.114428578297
16,0.114428578297
18,0.114428578297


In [69]:
import pandas as pd
df = pd.read_csv('bank-classification.csv')
df = df.sample(frac=1, replace=False, random_state=42)
print(df[:5])

          id  birth_date          job  marital    education  default housing  \
32884  32885  1952-07-15   technician  married  high.school       no      no   
3169    3170  1953-06-30      unknown  married      unknown  unknown     yes   
32206  32207  1976-09-12  blue-collar  married     basic.9y       no      no   
9403    9404  1972-04-10       admin.  married  high.school       no      no   
14020  14021  1981-05-25    housemaid  married  high.school       no     yes   

      loan contact_date    contact  campaign  pdays  previous     poutcome  \
32884  yes   2009-05-04   cellular         1    999         1      failure   
3169    no   2008-05-08  telephone         2    999         0  nonexistent   
32206   no   2009-05-08   cellular         1    999         1      failure   
9403    no   2008-06-27  telephone         4    999         0  nonexistent   
14020   no   2008-07-25   cellular         2    999         0  nonexistent   

             y  
32884  unknown  
3169   unknown  

In [71]:
train_df = df.loc[df['y'] != 'unknown']
test_df = df.loc[df['y'] == 'unknown']

In [72]:
train_df.get(['id', 'y']).groupby(['y']).count()

,id
y,
no,18419
yes,2380


In [49]:
train_df.get(['id', 'job']).groupby(['job']).count()

,id
job,
admin.,5284
blue-collar,4708
entrepreneur,745
housemaid,522
management,1485
retired,874
self-employed,699
services,1995
student,444


In [50]:
train_df.get(['id', 'education']).groupby(['education']).count()

,id
education,
basic.4y,2154
basic.6y,1170
basic.9y,3016
high.school,4781
illiterate,7
professional.course,2592
university.degree,6192
unknown,887


In [51]:
train_df.get(['id', 'marital']).groupby(['marital']).count()

,id
marital,
divorced,2347
married,12512
single,5896
unknown,44


In [73]:
import numpy as np
import matplotlib.pyplot as plt

def get_year(date):
    year_str = date[:4]
    year = int(year_str)
    return year

def get_input_data(df):
    ages = np.array([get_year(contact) - get_year(birth) for contact, birth in
                     zip(df['contact_date'], df['birth_date'])])
    return np.array([ages]).T, np.array(df['y'])

In [78]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(class_weight='balanced')
X_train, y_train = get_input_data(train_df)
print(X_train, y_train)

[[33]
 [36]
 [58]
 ..., 
 [58]
 [37]
 [29]] ['no' 'no' 'no' ..., 'no' 'no' 'no']


In [79]:
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [97]:
from pandas import DataFrame

def return_submission(classifier, path, test_df):
    X_test, _ = get_input_data(test_df)
    classes = classifier.classes_
    predictions = classifier.predict_proba(X_test)
    results_pd = DataFrame({'id': test_df['id'], 'y': predictions[::, 1]})
    results_pd = results_pd.sort_values('id')
    results_pd.to_csv(path, index=False)

return_submission(classifier, 'my_submission.csv', test_df)

In [98]:
!head my_submission.csv

id,y
2,0.4936496416243267
4,0.40885558836645464
5,0.46670458555999056
7,0.5637336969834605
10,0.5444788722692732
12,0.5444788722692732
14,0.4936496416243267
16,0.47730806481192856
18,0.4093685196913506
